In [0]:
-- Read each file into a named DataFrame using the parameterized base file path
CREATE OR REPLACE TEMP VIEW employees_branch_a
USING CSV
OPTIONS (path = 'file:/Workspace/Users/sleekdatalabs@gmail.com/oms-databricks/00_sample_data/employees_branch_a.csv', header = "true");

CREATE OR REPLACE TEMP VIEW employees_branch_b
USING CSV
OPTIONS (path = 'file:/Workspace/Users/sleekdatalabs@gmail.com/oms-databricks/00_sample_data/employees_branch_b.csv', header = "true");

CREATE OR REPLACE TEMP VIEW job_history_branch_a
USING CSV
OPTIONS (path = 'file:/Workspace/Users/sleekdatalabs@gmail.com/oms-databricks/00_sample_data/job_history_branch_a.csv', header = "true");

CREATE OR REPLACE TEMP VIEW job_history_branch_b
USING CSV
OPTIONS (path = 'file:/Workspace/Users/sleekdatalabs@gmail.com/oms-databricks/00_sample_data/job_history_branch_b.csv', header = "true");


In [0]:
-- Display a few rows from Temp View (you can use the same for all 4 tables)
SELECT * FROM employees_branch_a LIMIT 5;

In [0]:
-- Typecasting employees
CREATE OR REPLACE TEMP VIEW employees_branch_a_cast AS
SELECT 
    employee_id,
    name,
    CAST(age AS INT) AS age,
    CAST(salary AS DOUBLE) AS salary,
    department,
    CAST(join_date AS DATE) AS join_date,
    branch,
    gender
FROM employees_branch_a;

CREATE OR REPLACE TEMP VIEW employees_branch_b_cast AS
SELECT 
    employee_id,
    name,
    CAST(age AS INT) AS age,
    CAST(salary AS DOUBLE) AS salary,
    department,
    CAST(join_date AS DATE) AS join_date,
    branch,
    gender
FROM employees_branch_b;

-- Typecasting job_history
CREATE OR REPLACE TEMP VIEW job_history_branch_a_cast AS
SELECT 
    employee_id,
    role,
    CAST(start_date AS DATE) AS start_date,
    CAST(end_date AS DATE) AS end_date
FROM job_history_branch_a;

CREATE OR REPLACE TEMP VIEW job_history_branch_b_cast AS
SELECT 
    employee_id,
    role,
    CAST(start_date AS DATE) AS start_date,
    CAST(end_date AS DATE) AS end_date
FROM job_history_branch_b;

In [0]:
-- Union the employees and job history
CREATE OR REPLACE TEMP VIEW employees_union AS
SELECT * FROM employees_branch_a_cast
UNION ALL
SELECT * FROM employees_branch_b_cast;

CREATE OR REPLACE TEMP VIEW job_history_union AS
SELECT * FROM job_history_branch_a_cast
UNION ALL
SELECT * FROM job_history_branch_b_cast;

In [0]:
-- Join the unioned employees with the unioned job history
CREATE OR REPLACE TEMP VIEW joined_data AS
SELECT 
    e.*,
    j.role,
    j.start_date,
    j.end_date
FROM employees_union AS e
JOIN job_history_union AS j
ON e.employee_id = j.employee_id;

In [0]:
-- Filter rows with salary more than 50000
CREATE OR REPLACE TEMP VIEW filtered_data AS
SELECT *
FROM joined_data
WHERE salary > 50000;

In [0]:
-- Select only relevant columns
CREATE OR REPLACE TEMP VIEW selected_data AS
SELECT employee_id, name, age, salary, department, join_date, gender, role, start_date, end_date
FROM filtered_data;

In [0]:
-- Sort based on employee_id and start_date (desc)
CREATE OR REPLACE TEMP VIEW sorted_data AS
SELECT *
FROM selected_data
ORDER BY employee_id ASC, start_date DESC;

In [0]:
-- Deduplicate to keep the latest active role for each employee
CREATE OR REPLACE TEMP VIEW deduped_data AS
SELECT *
FROM (SELECT ROW_NUMBER() OVER (PARTITION BY employee_id ORDER BY start_date DESC) AS rn, *
      FROM sorted_data) tmp
WHERE rn = 1;

In [0]:
-- Calculate years_in_service using join_date and current date
CREATE OR REPLACE TEMP VIEW data_with_years_in_service AS
SELECT *, 
       FLOOR(DATEDIFF(CURRENT_DATE(), join_date) / 365) AS years_in_service
FROM deduped_data;

In [0]:
-- Drop rows where the role is Null
CREATE OR REPLACE TEMP VIEW cleaned_data AS
SELECT *
FROM data_with_years_in_service
WHERE role IS NOT NULL;

-- Replace Null values in age with 0
CREATE OR REPLACE TEMP VIEW cleaned_data_with_nulls_filled AS
SELECT 
    employee_id,
    name,
    COALESCE(age, 0) AS age,
    salary,
    department,
    join_date,
    gender,
    role,
    start_date,
    end_date,
    years_in_service
FROM cleaned_data;

In [0]:
-- Replace 'M' with 'Male', 'F' with 'Female', and any other value with 'Other' in the gender column
CREATE OR REPLACE TEMP VIEW cleaned_data_final AS
SELECT 
    employee_id,
    name,
    age,
    salary,
    department,
    join_date,
    CASE 
        WHEN gender = 'M' THEN 'Male'
        WHEN gender = 'F' THEN 'Female'
        ELSE 'Other'
    END AS gender,
    role,
    start_date,
    end_date,
    years_in_service
FROM cleaned_data_with_nulls_filled;

In [0]:
-- Calculate the average, minimum, and maximum salary for each role
CREATE OR REPLACE TEMP VIEW aggregated_data AS
SELECT role, 
       AVG(salary) AS avg_salary,
       MIN(salary) AS min_salary,
       MAX(salary) AS max_salary
FROM cleaned_data_final
GROUP BY role;

In [0]:
-- We are just displaying the data here, but temp view can be loaded into a table as shown in the next cell
select * from aggregated_data limit 5;

In [0]:
-- Write cleaned_data_final as employee_details table
-- CREATE OR REPLACE TABLE employee_details AS
-- SELECT *
-- FROM cleaned_data_final;

-- Write aggregated_data as employee_role_summary table
-- CREATE OR REPLACE TABLE employee_role_summary AS
-- SELECT *
-- FROM aggregated_data;
